# Mackey and Glass Work-Precision Diagrams
### David Widmann, Chris Rackauckas

# Model

We study a model of blood production that was published by M. C. Mackey and L. Glass in "Oscillation and chaos in physiological control systems", 1977, and is given by
```math
\begin{equation}
 y'(t) = \frac{0.2y(t-14)}{1 + y(t-14)^{10}} - 0.1y(t)
\end{equation}
```
for ``t \in [0, 500]`` with history function ``\phi(t) = 0.5`` for ``t \leq 0``. It is test problem A1 of W. H. Enright and H. Hayashi, "The evaluation of numerical software for delay differential equations", 1997.

# Setup

In [ ]:
using DelayDiffEq, DiffEqDevTools, DiffEqProblemLibrary.DDEProblemLibrary
DDEProblemLibrary.importddeproblems()

const prob = DDEProblemLibrary.prob_dde_DDETST_A1
const sol = solve(prob, MethodOfSteps(Vern9()); dtmax = 0.1, reltol = 1e-14, abstol = 1e-14)
const test_sol = TestSolution(sol)

using Plots
gr()
plot(sol)

In [ ]:
function buildWorkPrecisionSet(algs, abstols, reltols; kwargs...)
    setups = [Dict(:alg => MethodOfSteps(alg)) for alg in algs]
    names = nameof.(typeof.(algs))

    WorkPrecisionSet(prob, abstols, reltols, setups;
                     names = names, appxsol = test_sol, maxiters = Int(1e5), kwargs...)
end

## Low order RK methods

### High tolerances

First we test final error estimates of continuous RK methods of low order at high tolerances. `OwrenZen4`, `OwrenZen5`, and `RK4` yield the best error estimates.

In [ ]:
abstols = @. 1.0 / 10.0^(4:7)
reltols = @. 1.0 / 10.0^(1:4)
algs = [BS3(), Tsit5(), RK4(), DP5(), OwrenZen3(), OwrenZen4(), OwrenZen5()]

wp = buildWorkPrecisionSet(algs, abstols, reltols; error_estimate = :final)
plot(wp)

Next we test average interpolation errors:

In [ ]:
abstols = @. 1.0 / 10.0^(4:7)
reltols = @. 1.0 / 10.0^(1:4)
algs = [BS3(), Tsit5(), RK4(), DP5(), OwrenZen3(), OwrenZen4(), OwrenZen5()]

wp = buildWorkPrecisionSet(algs, abstols, reltols; error_estimate = :L2)
plot(wp)

As before, `OwrenZen4` and `OwrenZen5` perform well over the whole range of investigated tolerances.

### Low tolerances

We repeat our tests with low tolerances.

In [ ]:
abstols = @. 1.0 / 10.0^(8:11)
reltols = @. 1.0 / 10.0^(5:8)
algs = [BS3(), Tsit5(), RK4(), DP5(), OwrenZen3(), OwrenZen4(), OwrenZen5()]

wp = buildWorkPrecisionSet(algs, abstols, reltols; error_estimate = :final)
plot(wp)

And once again we also test the interpolation errors:

In [ ]:
abstols = @. 1.0 / 10.0^(8:11)
reltols = @. 1.0 / 10.0^(5:8)
algs = [BS3(), Tsit5(), RK4(), DP5(), OwrenZen3(), OwrenZen4(), OwrenZen5()]

wp = buildWorkPrecisionSet(algs, abstols, reltols; error_estimate = :L2)
plot(wp)

Apparently `Tsit5` and `DP5` perform quite well at low tolerances, but only `OwrenZen5`, `OwrenZen4` and `RK4` achieve interpolation errors of around 1e-9.

## Lazy interpolants

### High tolerances

We repeat our tests with the Verner methods which, in contrast to the methods above, use lazy interpolants. As reference we include `OwrenZen4`.

In [ ]:
abstols = @. 1.0 / 10.0^(4:7)
reltols = @. 1.0 / 10.0^(1:4)
algs = [Vern6(), Vern7(), Vern8(), Vern9(), OwrenZen4()]

wp = buildWorkPrecisionSet(algs, abstols, reltols; error_estimate = :final)
plot(wp)

And we obtain the following interpolation errors:

In [ ]:
abstols = @. 1.0 / 10.0^(4:7)
reltols = @. 1.0 / 10.0^(1:4)
algs = [Vern6(), Vern7(), Vern8(), Vern9(), OwrenZen4()]

wp = buildWorkPrecisionSet(algs, abstols, reltols; error_estimate = :L2)
plot(wp)

`Vern6`, `Vern7`, and `Vern9` are outperformed by `OwrenZen4`.

### Low tolerances

Again, we repeat our tests at low tolerances.

In [ ]:
abstols = @. 1.0 / 10.0^(8:11)
reltols = @. 1.0 / 10.0^(5:8)
algs = [Vern6(), Vern7(), Vern8(), Vern9(), OwrenZen4()]

wp = buildWorkPrecisionSet(algs, abstols, reltols; error_estimate = :final)
plot(wp)

In [ ]:
abstols = @. 1.0 / 10.0^(8:11)
reltols = @. 1.0 / 10.0^(5:8)
algs = [Vern6(), Vern7(), Vern8(), Vern9(), OwrenZen4()]

wp = buildWorkPrecisionSet(algs, abstols, reltols; error_estimate = :L2)
plot(wp)

`Vern6`, `Vern7`, and `Vern9` show similar results at low tolerances, and perform even better than `OwrenZen4`.

In [ ]:
using DiffEqBenchmarks
DiffEqBenchmarks.bench_footer(WEAVE_ARGS[:folder], WEAVE_ARGS[:file])